In [1]:
# Do all necessary preprocessing, calling prepro.py
import utils
from utils import *
importlib.reload(utils)
from utils import *

Using TensorFlow backend.


In [2]:
# Load data
X, X_test_original, y = load_data() 
y = y.ravel()
scores = np.array([])
xtrain = X  # For andreas cross validation
ytrain = y

In [ ]:
# # Isolation Forest in case we need it: 
# # Fit an isolation forest
# clf = IsolationForest(n_estimators=n_estimators)
# clf.fit(X_new)
# clf.decision_function(X_new)
# inlier_indices = np.where(clf.predict(X_new) == 1)[0]
# print("Size of inliers:", inlier_indices.shape)
# X_inliers = X_new[inlier_indices]
# y_inliers = y_[inlier_indices]
# X_test_inliers = X_test_new # Do not remove outliers from test dataset
# y_test_inliers = y_test
# print("Inliers shapes:", X_inliers.shape, y_inliers.shape)

# SMOTEClassifier(SMOTE(), xgb.XGBClassifier())

# Andreas Optimal Model:

In [ ]:
# Simple pipeline with grid search

steps = [("scaler", preprocessing.StandardScaler()), ("classifier", svm.SVC())]

pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf"],
              "classifier__gamma": ["auto"],
              "classifier__C": [0.1, 0.5, 1, 5, 10, 50, 100],
              "classifier__degree": [1, 2, 3, 4, 5, 6],
              "classifier__class_weight": ["balanced"]
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'balanced_accuracy')

# grid.fit(xtrain.values, ytrain.values.ravel())

# 0.7015740740740741
# {'classifier__C': 0.5, 'classifier__class_weight': 'balanced', 'classifier__degree': 1, 'classifier__gamma': 'auto', 'classifier__kernel': 'rbf'}


# Keras NN Grid Search

In [14]:
# dropout in hidden layers with weight constraint
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(, input_dim=1000, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(60, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.1))
    model.add(Dense(3, activation='sigmoid'))
    # Compile model
#     sgd = SGD(lr=0.02, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
# One hot encode data
# enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
# y_enc = enc.fit_transform(y)
y_enc = np.zeros((y.shape[0], 3))
y_enc[np.arange(y.shape[0]), y] = 1

estimators = []
estimators.append(('standardize', preprocessing.StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=3, shuffle=True)
cw = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
class_weight = {0: cw[0], 1: cw[1], 2: cw[2]}
def score(estimator, X_, y_):
    pred = estimator.predict(X_)
    y_normal = np.argmax(y_, axis=1)
    BMAC = balanced_accuracy_score(y_normal, pred)
    return BMAC
results = cross_val_score(pipeline, X, y_enc, cv=kfold, verbose=2, scoring=score, 
                          fit_params={'mlp__class_weight': class_weight})
print(results)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'IsolationForest(behaviour='old', bootstrap=False, contamination='legacy',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=None, verbose=0, warm_start=False)' (type <class 'sklearn.ensemble.iforest.IsolationForest'>) doesn't

In [6]:
pred = pipeline.predict(X)
BMAC = balanced_accuracy_score(y, pred)
# print(BMAC)
print(pred)

4800/4800 [==============================] - 0s 67us/step
[0 0 2 ... 1 1 2]


In [11]:

y_enc = np.zeros((y.shape[0], 3))
y_enc[np.arange(y.shape[0]), y] = 1
print(y_enc)
print(y)
np.argmax(y_enc, axis=1)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[1 0 2 ... 1 1 2]


array([1, 0, 2, ..., 1, 1, 2])

# Sklearn NN grid search (not working)

In [ ]:
steps = [("scaler", preprocessing.StandardScaler()), ("classifier", SMOTEClassifier(SMOTE(), MLPClassifier()))]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__solver": ['lbfgs'],
              "classifier__alpha": [1e-5, 1e-3, 0.1, 0.3, 0.5],
              "classifier__hidden_layer_sizes": [(30, 30), (80, 80), (150, 150)],
              "classifier__activation": ['relu', 'logistic'],
              "classifier__learning_rate": ["invscaling", "constant"], 
              "classifier__learning_rate_init": [0.0005, 0.001, 0.005, 0.01], 
              "classifier__beta_1": [0.8, 0.9, 0.999], 
              "classifier__beta_2": [0.8, 0.9, 0.999], 
              "classifier__early_stopping": [True], 
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = 'balanced_accuracy', verbose=1)

grid.fit(X, y)

# SKlearn NN and Andreas Ensemble

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)

mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,
                       hidden_layer_sizes=(150,150), random_state=1,
                        activation='relu', learning_rate='invscaling', beta_1=0.9, 
                          beta_2=0.999, learning_rate_init=0.001, early_stopping=True,
                          momentum=0.9, shuffle=True, epsilon=1e-08, 

                      )
andreas_svm = svm.SVC(C=0.5, class_weight='balanced', degree=1, gamma='auto', kernel='rbf')
eclf = VotingClassifier(estimators=[('svm', andreas_svm), ('mlp', mlp)], voting='hard')

steps = [("scaler", preprocessing.StandardScaler()), ("classifier", eclf)]
pipeline = Pipeline(steps = steps)

score = cross_val_score(pipeline, X, y, cv=kf, scoring=make_scorer(balanced_accuracy_score), verbose=2)
print("Score:", score)

# Ensemble score: Score: [0.66481481 0.64861111 0.67824074 0.6462963  0.65231481]
# Andreas SVM score: Score: [0.69907407 0.70740741 0.69583333 0.67962963 0.70416667]
# Result: Ensemble MLP (no SMOTE) + SVM is bad, and no SMOTE 